# Cytnx-Tutorial1
This tutorial is aimed for those who are familiar with using python doing tensor network calculation. Most of the examples are based on: https://www.tensors.net/p-tutorial-1



# Some Questions and Comments
Q: I know that cytnx.Tensor() is easy to transform to np.array(); however, can we do the reverse opeartions, i.e. tranform np.array() to cytnx.Tensor()? Since constructing array using numpy is still more convenient than using Cytnx (e.g. we can construct 3x3 matrix by directy typing the element in numpy), if we can convert np.array() to cytnx.Tensor(), it will be great. 

A: cytnx.from_numpy()

Q: We can transform cytnx.Tensor() to cyx.CyTensor(). However, can we do the reverse thing? Since only cytnx.Tensor() can compare with np.array(), it would be inconvenint if cyx.CyTensor() cannot transfrom to cytnx.Tensor() (See the last cell).

A: cytnx.Tensor and cyx.CyTensor serves different purpose, Tensor is like numpy array equivalence, while CyTensor is specifically designed to handle Tensor network algorithm
In the case where the system does not explicitly have symmetry enforced, Tensor and CyTensor seems to be redundant, but in the system with symmetries (as well as if we consider the Fermionic system, it will be completely different)

Q: Easy way to do the same thing as np.random.rand()?

A: cytnx.from_numpy(np.random.rand())


Q: See the last cell: How to test whether two cyx.CyTensor are almost the same (don't need to be A == B)?

A: Ahh... My answer is use get_block(), which can transform cyx.CyTensor to cytnx.Tensor(), and then we can do the subtraction operation on them.

In [2]:
import numpy as np
import scipy as sp
from scipy import linalg as LA # differ from cytnx.linalg
import sys
sys.path.append("/usr/local/")
import cytnx
from cytnx import cytnx_extension as cyx

- If we have a function cytnx.eye(), it will be more convenient to create identity matrix.

A: cytnx.from_numpy(np.eye()) XDDD

In [3]:
##### Lets initialize some tensors in Python/Numpy
import numpy as np
### tensor with randomly generated entries, order 3, dims: 2-by-3-by-4
A = np.arange(24).reshape(2,3,4)
Act = cytnx.arange(24).reshape(2,3,4)
# Atest = cytnx.from_numpy(A)
# print(Atest)
# print(type(A-Act))
print(LA.norm(A-Act))
# Act = np.array(A)
# print(Act)

### identity matrix, order 2, dims: 5-by-5 
B = np.eye(5,5)
Bct = cytnx.zeros([5,5])
for i in range(5):
    Bct[i,i] = 1
    
print(LA.norm(B- Bct))
### tensor of 1's, order 4, dims: 2-by-4-by-2-by-4
C = np.ones((2,4,2,4))
Cct = cytnx.ones([2,4,2,4])
print(LA.norm(C-Cct))

### matrix of 0's, order 2, dims: 3-by-5
D = np.zeros((3,5))
Dct = cytnx.zeros([3,5])
print(LA.norm(D-Dct))




0.0
0.0
0.0
0.0


# Bug: 
- Even though Atilda and AtildaCt look the same, (Atilda - AtildaCt) is not zero.
- Using cytnx.from_numpy(Atilda) seems to have a bug (due to transpose()? ).


In [4]:
##### Ex.1.2(a):Transpose
A = np.arange(2**3).reshape(2,2,2)
Atilda = A.transpose(1,2,0)

Act = cytnx.arange(2**3).reshape(2,2,2)
AtildaCt = Act.permute(1,2,0)
# print(A)
# print(Act)
# print(LA.norm(A-Act))


# print('Atilda = ', Atilda)
# print('AtildaCt = ', AtildaCt)

# print('Atilda - AtildaCt = ', Atilda - AtildaCt)
# print(LA.norm(Atilda - AtildaCt))

#### Another way
print(Atilda)
AtildaClone = Atilda.copy()
Atest = cytnx.from_numpy(AtildaClone) # good
Atest2 = cytnx.from_numpy(Atilda) #bad

# print(Atest)
# print(Atilda)
# print(Atest2)

# print(Atest -AtildaCt)

dA = LA.norm(Atest - AtildaCt)
print(dA)





[[[0 4]
  [1 5]]

 [[2 6]
  [3 7]]]
0.0


In [5]:
##### Ex.1.2(b):Reshape
B = np.arange(64).reshape(4,4,4)
Btilda = B.reshape(4,4**2)
Bct = cytnx.arange(64). reshape(4,4,4)
BtildaCt = Bct.reshape(4,4**2)
print(LA.norm(Btilda-BtildaCt))

0.0


In [6]:
##### Ex.1.3(a): Binary Tensor Contraction
d = 2
A = np.arange(16).reshape(d,d,d,d)  
B = np.arange(16).reshape(d,d,d,d)  

Ap  = A.transpose(0,2,1,3);  Bp = B.transpose(0,3,1,2)
App = Ap.reshape(d**2,d**2); Bpp = Bp.reshape(d**2,d**2)
Cpp = App @ Bpp;             C   = Cpp.reshape(d,d,d,d)

##### Using Cytnx
Act = cytnx.arange(16).reshape(d,d,d,d)  
Bct = cytnx.arange(16).reshape(d,d,d,d)
ApCt  = Act.permute(0,2,1,3);  BpCt = Bct.permute(0,3,1,2)
AppCt = ApCt.reshape(d**2,d**2); BppCt = BpCt.reshape(d**2,d**2)
CppCt = cytnx.linalg.Matmul(AppCt, BppCt);             Cct   = CppCt.reshape(d,d,d,d)


print(LA.norm(C - Cct))

0.0


# Bug (Solved)

Ect is not equal to np.array(Ect) after Ect.permute()


In [9]:
##### Ex.1.5(b): Contraction using ncon
from ncon import ncon # Need ncon.py in the directory

d = 3
A = np.arange(27).reshape(d,d,d); B = np.arange(81).reshape(d,d,d,d)
C = np.arange(27).reshape(d,d,d); D = np.arange(9).reshape(d,d)

TensorArray = [A,B,C,D]

IndexArray = [[1,-2,2],[-1,1,3,4],[5,3,2],[4,5]]

E = ncon(TensorArray,IndexArray)

######### Using Cytnx

Act = cytnx.arange(27).reshape(d,d,d); Bct = cytnx.arange(81).reshape(d,d,d,d)
Cct = cytnx.arange(27).reshape(d,d,d); Dct = cytnx.arange(9).reshape(d,d)
Act = cyx.CyTensor(Act,2)
Act.set_labels([1,-2,2])
Bct = cyx.CyTensor(Bct,2)
Bct.set_labels([-1,1,3,4])
Cct = cyx.CyTensor(Cct,2)
Cct.set_labels([5,3,2])
Dct = cyx.CyTensor(Dct,2)
Dct.set_labels([4,5])

# Act.print_diagram()
# Bct.print_diagram()
# Cct.print_diagram()
# Dct.print_diagram()
# Result = cyx.Contract(A)

Ect = cyx.Contract(cyx.Contract(Act,Bct),cyx.Contract(Cct,Dct))
Ect.print_diagram()


Ect = Ect.get_block()
print(E)
print(Ect)
print(np.array(Ect)) # Now, Ect == np.array(Ect)
Ect = Ect.permute(1,0)
Ect = Ect.contiguous()
print(Ect) ### But now, Ect != np.array(Ect)
print(np.array(Ect))

print(E) 
dE = LA.norm(E.T-Ect) # This will be zero
print(dE)

print('Ect[1,0].item() - E[1,0] = ', Ect[1,0].item() - E[1,0]) # Even though  Ect[1,0].item() - E[1,0] = 0
print(E-Ect) # this one is still not equal to zero matrix
print(np.array(Ect) -Ect) # But this is zero

-----------------------
tensor Name : 
tensor Rank : 2
block_form  : false
is_diag     : False
on device   : cytnx device: CPU
            -------------      
           /             \     
    -2 ____| 3         3 |____ -1 
           \             /     
            -------------      
[[ 2723544  3309660  3895776]
 [ 6546420  8274150 10001880]
 [10369296 13238640 16107984]]

Total elem: 9
type  : Double (Float64)
cytnx device: CPU
Shape : (3,3)
[[2.72354e+06 6.54642e+06 1.03693e+07 ]
 [3.30966e+06 8.27415e+06 1.32386e+07 ]
 [3.89578e+06 1.00019e+07 1.61080e+07 ]]



[[ 2723544.  6546420. 10369296.]
 [ 3309660.  8274150. 13238640.]
 [ 3895776. 10001880. 16107984.]]

Total elem: 9
type  : Double (Float64)
cytnx device: CPU
Shape : (3,3)
[[2.72354e+06 3.30966e+06 3.89578e+06 ]
 [6.54642e+06 8.27415e+06 1.00019e+07 ]
 [1.03693e+07 1.32386e+07 1.61080e+07 ]]



[[ 2723544.  3309660.  3895776.]
 [ 6546420.  8274150. 10001880.]
 [10369296. 13238640. 16107984.]]
[[ 2723544  3309660  389577

In [18]:
A = np.ones([3,3,3])
print(LA.norm(A))
Act = cytnx.from_numpy(A)
# print(cytnx.linalg.Norm(Act))
print(cytnx.linalg.abs(Act))


5.196152422706632


AttributeError: module 'cytnx.cytnx.linalg' has no attribute 'abs'